# This is the Model and Steps for the Training (Should be fine to run code sequentially starting here)

In [1]:
from transformers import EncoderDecoderModel, BertTokenizer,  EncoderDecoderConfig, BertConfig, BertModel
import torch
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf

In [2]:
#Load smallest dataset
d = pd.read_csv("scb-mt-en-th-2020/apdf.csv")
d.head()

,en_text,th_text
0,FAR LEFT: Indonesian National Police Chief Tit...,(ซ้ายสุด) นายติโต คาร์นาเวียน ผู้บัญชาการตำรวจ...
1,(Pictured: A rocket believed to be a Hwasong m...,(ภาพ: จรวดซึ่งเชื่อว่าเป็นขีปนาวุธฮวาซองเช่นเด...
2,"(Pictured: From left, Indian Air Commander Ak ...",(ภาพ: จากซ้าย นายอัค โชราเชีย ผู้บัญชาการทางอา...
3,"(Pictured: Left to right, Australia’s Defence ...",(ภาพ: จากซ้ายไปขวา นางมารีส เพย์น รัฐมนตรีว่าก...
4,(Pictured: A man watches the live-stream trans...,(ภาพ: ชายคนหนึ่งชมการแพร่ภาพออกอากาศสดการปล่อย...


In [3]:
#Creating the EncoderDecoder Model with Multilingual BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
model.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [13]:
#Setting up the Optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [14]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [5]:
encoding

{'input_ids': tensor([[   101,    113,   1403,  43102,  20503, 110626,    114,   1417,  39376,
          80275, 111433,  30011,   1397,  87226,  62904, 111431,  31287,  77654,
          16000,   1420,  38468,  35933,  49097, 111427,  55749, 104034,  53123,
          30011,  55593,  22765,  31287,  46856, 111432, 111424,  44334,  95809,
          33178,  26027, 111433,  22123,  96106,  66756,    113,  85706,  91692,
          43102,  20503,    114,   1417,  39376, 111433,  22765,  16000,  89970,
          74483,   1450,  22123,  61888,   1452,  22765,  91692,  17344,   1420,
          38468,  35933,  49097, 111427,  55749, 104034,  53123,  30011,  55593,
          22765,  31287,  46856, 111432, 111424,  44334,  95809,  72245,  78045,
          28220,  49292,  49292,  26027,  33872,  15493,  62904,  20503,  31256,
          17344,  20507,  76062,   1436,  17344,  49097,  38468,   1418, 101983,
          87226,    100,   1454,  80814,  31287,  35933, 111432,  80814,  16000,
          8996

Converting to Labels

In [10]:
decoder_ids.shape

torch.Size([2, 99])

In [11]:
decoder_ids2 = decoder_ids[:, 1:]

In [12]:
decoder_ids2.shape

torch.Size([2, 98])

In [13]:
zeros = torch.zeros((2, 1), dtype=torch.long)
zeros.shape

torch.Size([2, 1])

In [14]:
decoder_ids2 = torch.cat((decoder_ids2, zeros), 1)
decoder_ids2.shape

torch.Size([2, 99])

In [23]:
outputs = model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_ids , labels=decoder_ids2, decoder_attention_mask=dec_attention_mask)
loss = outputs.loss
loss.backward()
optimizer.step()

NameError: name 'optimizer' is not defined

In [16]:
outputs

Seq2SeqLMOutput(loss=tensor(13.5336, grad_fn=<NllLossBackward>), logits=tensor([[[-12.8898, -12.7697, -12.6687,  ..., -13.1129, -12.2185, -14.0893],
         [-10.5901, -10.7593, -11.5879,  ...,  -9.8728, -11.5277, -11.5026],
         [-10.5575, -11.5543, -11.5371,  ..., -11.0609, -11.3144, -12.0492],
         ...,
         [-10.3748, -10.2326, -10.3485,  ...,  -8.7455, -10.8703, -11.2602],
         [ -9.6800, -10.1785, -10.2593,  ...,  -8.3523,  -9.8951,  -9.6492],
         [-11.4613, -11.6432, -11.6907,  ..., -10.0338, -11.6970, -12.1428]],

        [[-12.7297, -13.6542, -13.4775,  ..., -12.4260, -14.1338, -11.5806],
         [-15.5969, -16.8048, -15.7194,  ..., -15.5130, -15.8202, -16.0875],
         [-17.4295, -17.7423, -17.7470,  ..., -16.9000, -18.3410, -16.5259],
         ...,
         [-11.0719, -10.4177, -11.2195,  ..., -10.0091,  -9.9369, -10.4211],
         [ -9.9867,  -9.7612,  -9.4864,  ...,  -7.7125,  -8.2061,  -9.8370],
         [-10.9080, -10.6906, -10.6252,  ...,  -8.3

## Training loop 


In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

## Zach suggested steps:
1. load encoder
2. separately load decoder
3. Run batch through encoder
4. use outputs of encoder (hidden_states) as decoder inputs
5. manually compute softmax   

### Similar process as before; training decoder only

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
text_batch = d["th_text"].head(2).tolist()
text_batchdec = d["en_text"].head(2).tolist()
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
decoding = tokenizer(text_batchdec, return_tensors='pt', padding=True, truncation=True)
decoder_ids = decoding['input_ids']
dec_attention_mask = decoding['attention_mask']

In [19]:
encoder = BertModel.from_pretrained("bert-base-multilingual-cased")
decoder = BertModel.from_pretrained("bert-base-multilingual-cased", is_decoder = True, add_cross_attention = True)

Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.1.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.1.crossattention.self.value.weight', 'bert.encoder.laye

In [5]:
enc_output = encoder(**encoding)
enc_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4598, -0.4631,  0.2443,  ...,  0.0354, -0.1929, -0.2172],
         [-0.5433, -0.0899,  0.0457,  ...,  0.9347,  0.3031, -0.4014],
         [ 0.2243, -0.4547, -0.0205,  ...,  0.3991,  0.7170,  0.1103],
         ...,
         [ 0.4330, -0.2473,  0.3002,  ...,  0.6456, -0.1040, -0.2901],
         [ 0.3162, -0.2680,  0.6232,  ...,  0.5241,  0.0563, -0.1591],
         [ 0.3500, -0.1957,  0.6513,  ...,  0.4272,  0.1462, -0.1288]],

        [[-0.1276,  0.0668, -0.1042,  ...,  0.1553,  0.1080, -0.1023],
         [-0.3135,  0.1371, -0.0257,  ...,  0.8119,  0.1915, -0.4935],
         [-0.2212, -0.0821,  0.5071,  ...,  0.3953, -0.0236,  0.0105],
         ...,
         [-0.3313, -0.4243,  0.4796,  ...,  1.6601,  0.7521, -0.0857],
         [-0.2170, -0.1398, -0.1424,  ...,  1.1104, -0.2227, -0.3726],
         [ 0.1812, -0.3896,  0.1867,  ..., -0.1006, -0.0927, -0.0226]]],
       grad_fn=<NativeLayerNormBackward>), pooler_out

In [6]:
last_hidden_states = enc_output.last_hidden_state
last_hidden_states

tensor([[[ 0.4598, -0.4631,  0.2443,  ...,  0.0354, -0.1929, -0.2172],
         [-0.5433, -0.0899,  0.0457,  ...,  0.9347,  0.3031, -0.4014],
         [ 0.2243, -0.4547, -0.0205,  ...,  0.3991,  0.7170,  0.1103],
         ...,
         [ 0.4330, -0.2473,  0.3002,  ...,  0.6456, -0.1040, -0.2901],
         [ 0.3162, -0.2680,  0.6232,  ...,  0.5241,  0.0563, -0.1591],
         [ 0.3500, -0.1957,  0.6513,  ...,  0.4272,  0.1462, -0.1288]],

        [[-0.1276,  0.0668, -0.1042,  ...,  0.1553,  0.1080, -0.1023],
         [-0.3135,  0.1371, -0.0257,  ...,  0.8119,  0.1915, -0.4935],
         [-0.2212, -0.0821,  0.5071,  ...,  0.3953, -0.0236,  0.0105],
         ...,
         [-0.3313, -0.4243,  0.4796,  ...,  1.6601,  0.7521, -0.0857],
         [-0.2170, -0.1398, -0.1424,  ...,  1.1104, -0.2227, -0.3726],
         [ 0.1812, -0.3896,  0.1867,  ..., -0.1006, -0.0927, -0.0226]]],
       grad_fn=<NativeLayerNormBackward>)

In [17]:
encoder.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Drop

In [20]:
#add dimension to last_hidden
decoder(input_ids = decoder_ids, labels = decoder_ids2, output_hidden_states = last_hidden_states)

TypeError: forward() got an unexpected keyword argument 'labels'

In [8]:
decoder(enc_output)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'size'

## =========================================================================================================


### Using the Trainer

In [ ]:
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=pd.DataFrame(tokenizer(d["th_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True), tokenizer(d["en_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True)),         # training dataset
    eval_dataset=pd.DataFrame(tokenizer(d["th_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True), tokenizer(d["en_text"].head(1000).tolist(), return_tensors='pt', padding=True, truncation=True)),         # training dataset,            # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
tokenizer(d["en_text"].head(100).tolist())

In [ ]:
tokenizer.decode(input_ids[0])

In [ ]:
tokenizer.decode(decoder_ids[0])

In [ ]:
outputs

In [ ]:
lens = []
for i in d["th_text"].head(100):
    lens.append(len(i))
max_length = max(lens)

In [ ]:
#Attempt to use class example for BERT enc/dec
def mt_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    tensors = tf.keras.layers.Input(shape=(max_length,))
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    input_ids = tensors
    
    
    
    # Note: Bert layer from Hugging Face returns two values: sequence ouput, and pooled output. Here, we only want
    # the former. (See https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel) 
    
    bert_layer = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
    
    # Freeze layers, i.e. only train number of layers specified, starting from the top
    if not train_layers == -1:
        
        retrain_layers = []
    
        for retrain_layer_number in range(train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)

        for w in bert_layer.weights:
            if not any([x in w.name for x in retrain_layers]):
                w._trainable = False

        # End of freezing section
    bert_sequence = bert_layer(input_ids=input_ids, decoder_input_ids=input_ids)[0]
    
    print('Let us check the shape of the BERT layer output:', bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dense = tf.keras.layers.Dropout(rate=0.1)(dense)
    
    pred = tf.keras.layers.Dense(21, activation='softmax', name='mt')(dense)
     
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
    
    losses = {
        "mt": keras.losses.SparseCategoricalCrossentropy(),
        }
    lossWeights = {"mt": 1.0
                  }
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss=losses, optimizer=optimizer, metrics=[bleu])
    
    
    model.summary()
    
    return model

In [ ]:
model = mt_model(max_length + 1, train_layers=-1, optimizer = tf.keras.optimizers.RMSprop)

model.fit(
    tokenizer(d["th_text"].head(100).tolist(), return_tensors='pt', padding=True), d["en_text"].head(100).tolist(), epochs = 10
)

In [ ]:
tokenizer(d["en_text"].head(100), return_tensors='tf', padding=True)

In [ ]:
d['en_text'].head(100).tolist()

In [ ]:
input_ids = torch.tensor(tokenizer.encode(testdata["th_text"][0], add_special_tokens=True)).unsqueeze(0)

In [ ]:
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)

In [ ]:
loss, logits = outputs.loss, outputs.logits

In [ ]:
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

In [ ]:
tokenizer.decode(generated[0])

### Exploring MT5 for translation

In [ ]:
from transformers import MT5Model, T5Tokenizer, TFMT5ForConditionalGeneration
import sentencepiece

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

In [ ]:
model = TFMT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
inputs = tokenizer('translate English to German: The house is wonderful.', return_tensors='tf').input_ids
labels = tokenizer('Das Haus ist wunderbar.', return_tensors='tf').input_ids

In [ ]:
outputs = model(inputs, labels=labels)

In [ ]:
loss = outputs.loss
loss

In [ ]:
logits = outputs.logits
logits

In [ ]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
inputs = tokenizer("summarize: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
inputs = tokenizer("translate English to Spanish: studies have shown that owning a dog is good for you ", return_tensors="tf").input_ids

In [ ]:
result = model.generate(inputs)

In [ ]:
result

In [ ]:
tokenizer.decode(result[0])

# Trainable Model



In [ ]:
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, EncoderDecoderModel, BertTokenizer, EncoderDecoderConfig, BertConfig, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, default_data_collator
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf
import torch
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [ ]:
metric = load_metric("bleu")

In [ ]:
# Use huggingface method to load
d = pd.read_csv("apdf.csv")

In [ ]:
datasets = load_dataset('csv', data_files='apdf.csv')

In [ ]:
datasets

In [ ]:
train_dataset = datasets["train"]

In [ ]:
train_dataset = train_dataset.select(range(100))
train_dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

In [ ]:
# In Example, not sure if to keep -- we'll see if this works
model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids("thai")

In [ ]:
print(model.config.decoder_start_token_id)

In [ ]:
column_names = datasets["train"].column_names

In [ ]:
#max_source_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['th_text']])
max_source_length = 512
max_target_length = 365
#max_target_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['en_text']])

In [ ]:
max_target_length

In [ ]:
def preprocess_function(examples):
        inputs = [ex for ex in examples["th_text"]]
        targets = [ex for ex in examples["en_text"]]
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding="max_length", truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding="max_length", truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        #if padding == "max_length" and data_args.ignore_pad_token_for_loss:
         #   labels["input_ids"] = [
          #      [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
           # ]
        # Was Label Previously
        model_inputs["labels"] = labels["input_ids"]
        #model_inputs["labels"] = labels["input_ids"]

        return model_inputs

In [ ]:
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=column_names,
        )

In [ ]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

In [ ]:
def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.decode(preds, skip_special_tokens=True)
       
        decoded_labels = tokenizer.decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

In [ ]:
data_collator = default_data_collator

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',
)

In [ ]:
trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        #eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

In [ ]:
trainer.train()

In [ ]:
train_dataset